# Lab 08 - Regularization

In previous labs we fitted and analyzed different algorithms for different hypothesis classes. In many of those, the chosen hypothesis was the one minimizing some cost function $\mathcal{F}_S\left(h\right)$ for some training set $S=\left\{\left(\mathbf{x}_i,y_i\right)\right\}^m_{i=1}$, and where $\mathcal{F}$ measures the goodness of $h$'s fit to $S$. 

Along side fitting a hypothesis, we have also discussed the richness and expressiveness of different hypothesis classes. For example, we have seen how the depth of a classification tree or the degree of a fitted polynomial influences prediction. We built the intuition of how the richness of the hypothesis class, through which the richness of the selected hypothesis, influences the bias-variance treade-off and the generalization error. 

The concept of **regularization** is of constraining the fitting process to enable the selection of complex models, but only if it is indeed "justified enough". Instead of minimizing only $\mathcal{F}$ we introduce an additional **regularization term** that depends on the tested hypothesis. We are searching for hypotheses that minimized the joint expression $$ h_S = \underset{h\in\mathcal{H}}{\text{argmin}} \,\, \mathcal{F}_S\left(h\right) + \lambda \mathcal{R}\left(h\right) $$  
where we select $\mathcal{R}$ in a way that measures the "complexity" of the hypothesis $h$. In this lab we focus on two modern regularization terms for regression: Lasso- and Ridge regressions.

In [15]:
import sys
sys.path.append("../")
from utils import *

## Loading Data and Fitting Models
To investigate how Lasso and Ridge regressions work we will use the [mtcars dataset](https://www.rdocumentation.org/packages/datasets/versions/3.6.2/topics/mtcars). This dataset was extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973-74 models).

*The `mpg` column, which stands for miles per gallon of fuel is the response value to be predicted.*

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge

np.random.seed(0)
X = pd.read_csv("../datasets/mtcars.csv").drop(columns=["model"]).dropna()

tr, te= train_test_split(X, test_size=0.4)
X_train, y_train, X_test, y_test = tr.loc[:, tr.columns != "mpg"], tr["mpg"], te.loc[:, te.columns != "mpg"], te["mpg"]

X.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


The Ridge and Lasso models minimize the joint loss of the $MSE$ and a regularization terms of the coefficient's vector size (norm). In the case of Ridge the norm is the $\ell_2$ Euclidean norm while in the case of Lasso it is the $\ell_1$ norm:
$$
\hat w^{ridge}_{\lambda} = \underset{w_0\in\mathbb{R}, w\in\mathbb{R}^d}{\text{argmin}} \Vert w_0 + Xw - y \Vert^2_2 +
    \lambda \Vert w \Vert^2_2
$$
$$
\hat w^{lasso}_{\lambda} = \underset{w_0\in\mathbb{R}, w\in\mathbb{R}^d}{\text{argmin}} \Vert w_0 + Xw - y \Vert^2_2\ +
    \lambda \Vert w \Vert_1
$$

Let us fit both models of Ridge and Lasso over the `mtcars` dataset for different values of the regularization parameter $\lambda$. For each value of $\lambda$ we will store the fitted coefficients as well as the losses they achieve (both the MSE and the regularization term values).




In [22]:
lambdas = 10**np.linspace(-3, 2, 100)

models = [dict(name="Lasso",
               model=lambda lam, x, y: Lasso(alpha=lam, normalize=True, max_iter=10000, tol=1e-4).fit(x, y),
               reg_penalty= lambda lam, w: lam*np.linalg.norm(w, ord=1)),
         dict(name="Ridge",
               model=lambda lam, x, y: Ridge(alpha=lam, normalize=True).fit(x, y),
               reg_penalty= lambda lam, w: lam*np.linalg.norm(w, ord=2))]


regressors = {}
for m in models:
    res = dict(coefs  = pd.DataFrame([], columns=list(X_train.columns),  index = lambdas),
               losses = pd.DataFrame([], columns=["mse", "reg", "loss"], index = lambdas))
   
    for lam in lambdas:
        model = m["model"](lam, X_train, y_train)
        res["coefs"].loc[lam, :] = model.coef_
        
        mse = mean_squared_error(y_test, model.predict(X_test))
        reg = m["reg_penalty"](lam, model.coef_)
        res["losses"].loc[lam, :] = [mse, reg, mse+reg]
        
    regressors[m["name"]] = res

c:\users\0erel\onedrive\documents\cs.bsc\semd\iml\iml.huji repo\venv\lib\site-packages\sklearn\linear_model\_base.py:145: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 

c:\users\0erel\onedrive\documents\cs.bsc\semd\iml\iml.huji repo\venv\lib\site-packages\sklearn\linear_model\_base.py:145: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a Stand

## Evaluating Results

Beginning with the Ridge regression, let us look at the losses and regularization path for each value of $\lambda$. For each such value of $\lambda$ we got a different model that minimized the joint loss function. We can see that as we increase $\lambda$ all coefficients go towards $0$. This is known as shrinkage of the coefficients. 

For $\lambda=100$, the penalty induced by the regularization term is so large that all coefficients are very close to zero, though for this dataset non is actually getting zero. Looking at the losses graph we see that the $\lambda$ achieving lowest joint loss (MSE and regularization together) over the test set is $\lambda\approx0.1047$.

In [18]:
coefs, losses = regressors["Ridge"].values()


fig = make_subplots(rows=2, cols=1, subplot_titles=[r'$\text{Regularization Path}$', r'$\text{Model Losses}$'], 
                    row_heights=[400,200], vertical_spacing=.1)

# Plot the regularization path for each feature
for i, col in enumerate(X_train.columns):
    fig.add_trace(go.Scatter(x=lambdas, y=coefs.loc[:, col], mode='lines', name=col, legendgroup="1"))


# Plot the losses graph and mark lambda with lowest loss
lam = np.argmin(losses.loc[:, 'loss'])
fig.add_traces([go.Scatter(x=lambdas, y=losses.loc[:, 'mse'], mode='lines', name="Fidelity Term - MSE", legendgroup="2"),
                go.Scatter(x=lambdas, y=losses.loc[:, 'reg'], mode='lines', name="Regularization Term", legendgroup="2"),
                go.Scatter(x=lambdas, y=losses.loc[:, 'loss'], mode='lines', name="Joint Loss", legendgroup="2"),
                go.Scatter(x=[lambdas[lam]], y=[losses.loc[:, 'loss'].values[lam]], mode='markers', showlegend=False,
                           marker=dict(size=8, symbol="x"), hovertemplate="Lambda: %{x}<extra></extra>")], 2, 1)

fig.update_layout(hovermode='x unified', margin=dict(t=50), 
                  legend=dict(tracegroupgap = 60),
                  title=r"$(1)\text{ Fitting Ridge Regression}$")\
   .update_xaxes(type="log")

Next, we look at the results of fitting a Lasso regression. Recall that the "only" difference between these two optimization problems is that Lasso uses the $\ell_1$ norm as the regularization term. By doing so it introduces **sparsity** to the solutions. Similar to the Ridge regression coefficients are shrinked towards zero, but due to the sparsity, we are more likely to get coefficients of *exactly* zero.

For example:
- For $\lambda=0.00722$ we observe the `cyl` feature getting a zero coefficient.
- For $\lambda=0.0327$ we observe the `drat` feature getting a zero coefficient.
- For $\lambda=1.353$ we observe the `wt` feature getting a zero coefficient, which at this point all features are fitted with a coefficient of zero.

The regularization parameter that yields the lowest joint loss over the test set is $\lambda=0.2983$.

In [19]:
coefs, losses = regressors["Lasso"].values()


fig = make_subplots(rows=2, cols=1, subplot_titles=[r'$\text{Regularization Path}$', r'$\text{Model Losses}$'], 
                    row_heights=[400,200], vertical_spacing=.1)

# Plot the regularization path for each feature
for i, col in enumerate(X_train.columns):
    fig.add_trace(go.Scatter(x=lambdas, y=coefs.loc[:, col], mode='lines', name=col, legendgroup="1"))


# Plot the losses graph and mark lambda with lowest loss
lam = np.argmin(losses.loc[:, 'loss'])
fig.add_traces([go.Scatter(x=lambdas, y=losses.loc[:, 'mse'], mode='lines', name="Fidelity Term - MSE", legendgroup="2"),
                go.Scatter(x=lambdas, y=losses.loc[:, 'reg'], mode='lines', name="Regularization Term", legendgroup="2"),
                go.Scatter(x=lambdas, y=losses.loc[:, 'loss'], mode='lines', name="Joint Loss", legendgroup="2"),
                go.Scatter(x=[lambdas[lam]], y=[losses.loc[:, 'loss'].values[lam]], mode='markers', showlegend=False,
                           marker=dict(size=8, symbol="x"), hovertemplate="Lambda: %{x}<extra></extra>")], 2, 1)

fig.update_layout(hovermode='x unified', margin=dict(t=50), 
                  legend=dict(tracegroupgap = 60),
                  title=r"$(2)\text{ Fitting Lasso Regression}$")\
   .update_xaxes(type="log")

## The Shrinkage of Coefficients
Let us compare between the fitted coefficients of the Least Squares (LS), the Lasso and the Ridge regressions. For the Lasso and Ridge regressions we will select the $\lambda$ achieving the lowest test error.


In [20]:
ls = LinearRegression().fit(X_train, y_train).coef_
ridge = regressors["Ridge"]["coefs"].iloc[np.argmin(regressors["Lasso"]["losses"].loc[:, "loss"])]
lasso = regressors["Lasso"]["coefs"].iloc[np.argmin(regressors["Lasso"]["losses"].loc[:, "loss"])]

coefs = np.array([ls, ridge, lasso])
pd.DataFrame(coefs, columns=list(X_train.columns), index=["LS", "Ridge", "Lasso"])

,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
LS,1.21023,0.018894,-0.02139,1.761205,-4.745438,0.9429,1.370546,1.142234,3.430979,-0.884874
Ridge,-0.277475,-0.005609,-0.008697,0.744696,-2.03097,0.185558,1.186673,1.215819,0.753781,-0.619353
Lasso,-0.728665,-0.0,-0.007548,0.0,-3.150145,0.0,0.310297,0.0,0.0,-0.0


In the following visualization of the coefficients under different models we can see the shrinkage of the coefficients. Notice that between the LS and Ridge models the overall size of the coefficients decreases. A similar outcome is seen between the LS and Lasso coefficients, with the difference that in the Lasso case many features are given a $0$ coefficient. In fact, only the `vs`, `cyl` and `wt` features are with non-zero coefficients. This shows how the Lasso regression also performs a type of feature selection.

In [21]:
fig = go.Figure(layout=go.Layout(title=r"$\text{(3) Regression Models Coefficients}$",
                                 xaxis=dict(range=[-1.1, 1.1], showticklabels=False, zeroline=False) ))

fig.add_annotation(x=-.8, y=6, text=r"$\text{LS Coefficients}$",   showarrow=False)
fig.add_annotation(x=0,   y=6, text=r"$\text{Ridge Coefficients}$", showarrow=False)
fig.add_annotation(x=.8,  y=6, text=r"$\text{Lasso Coefficients}$", showarrow=False)

for i, col in enumerate(X_train.columns):
    fig.add_trace(go.Scatter(x=[-1, 0, 1], y=coefs[:, i], mode='markers+lines', name=col, 
                             line={'dash':'dot'}))

fig.show()